GradientBoostingClassifier  -  Casey Kellett Whitwham



In [22]:
import importlib
import dataprep as dp
importlib.reload(dp)


from sklearn.metrics import accuracy_score,confusion_matrix

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier


import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
Xtrain,ytrain,Xtest,ytest,Xdev,ydev = dp.loadprocesseddata()

(24384, 12)
Train X,y shape:  (14609, 84) (14609,)
Test X,y shape:  (4876, 84) (4876,)
Dev X,y shape:  (4878, 84) (4878,)


In [24]:
def gridsearch(X, y,param_grid,model):
    grid = GridSearchCV(model, param_grid,
                       scoring = "accuracy",
                       cv = 5, verbose = 1,
                       return_train_score=True,n_jobs=-1)
    grid.fit(X,y)
    print("Grid search DONE!")
    return grid



def rungrid(X_dev, y_dev,y_train, paramgrid, model):
    grid = gridsearch(X_dev, y_dev, paramgrid, model)
    df_grid_result = pd.DataFrame(grid.cv_results_)
    columns_to_show = ['params', 'rank_test_score', 'mean_train_score', 'mean_test_score']
    df_grid_result = df_grid_result[columns_to_show]
    df_grid_result = df_grid_result.sort_values(by=['rank_test_score'], ascending=True)
    mean_ytrain_prepared = np.mean(y_train)
    mean_ytest_prepared = np.mean(y_dev)
    df_grid_result.rename(columns={'mean_test_score': 'mean_dev_score'}, inplace=True)
    df_grid_result['train_error_rate'] = - 100 * df_grid_result['mean_train_score'] / mean_ytrain_prepared
    df_grid_result['dev_error_rate'] = - 100 * df_grid_result['mean_dev_score'] / mean_ytest_prepared
    top = df_grid_result.iloc[0]["params"]

    return top

In [25]:
top_dt ={'criterion': 'entropy','max_depth': 10,'max_features': None, 'min_samples_leaf': 4,'min_samples_split': 10, 'class_weight': None}
dt_fin =  DecisionTreeClassifier(**top_dt)

In [26]:
ytrain = np.ravel(ytrain)
ydev = np.ravel(ydev)

In [27]:
#=== Gradient Param_Grid ===#

paramgrid_grad = {'n_estimators': [125,150,175],
                 'learning_rate': [0.01, 0.1, 0.5],
                 'loss': ['log_loss', 'exponential']}


In [30]:

top_dt_upd = top_dt.copy() 
top_dt_upd.pop('class_weight', None)
top_dt_upd.pop('criterion', None)


gradclass = GradientBoostingClassifier(**top_dt_upd)

top_grad = rungrid(Xdev, ydev, ytrain, paramgrid_grad, gradclass)
print("Top Params:", top_grad)

grad_fin = GradientBoostingClassifier(**top_grad, **top_dt_upd)

grad_fin.fit(Xtrain, ytrain)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
Grid search DONE!
Top Params: {'learning_rate': 0.5, 'loss': 'log_loss', 'n_estimators': 150}


GradientBoostingClassifier(learning_rate=0.5, max_depth=10, min_samples_leaf=4,
                           min_samples_split=10, n_estimators=150)

In [34]:
y_pred = grad_fin.predict(Xtest)
accuracy = accuracy_score(ytest, y_pred)
print("Test Accuracy:", accuracy)



Test Accuracy: 0.9319114027891715
